In [2]:
# Import library yang diperlukan
import pandas as pd

In [3]:
import pandas as pd

# Membaca data
df = pd.read_excel("C:/Users/hp/Downloads/Semester 5/Scrape Sinergia/Shopee/clean ulasan face up saja shopee.xlsx")
df.head()

,ulasan
0,terima kasih bonus
1,kemasan baru menarik coba enak lembut kotoran...
2,bagus poko recommended keseharian kerja double...
3,bner mengangkat kotoran wajah tanpa membuat ke...
4,lembut man


In [17]:
import pandas as pd
import re
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
from imblearn.over_sampling import SMOTE
from nltk.corpus import stopwords
import nltk
from textblob import TextBlob

# Download stopwords (run this once to ensure NLTK has the stopwords dataset)
nltk.download('stopwords')

# Preprocessing function
def preprocess_text(text, lang='indonesian'):
    # Lowercase the text
    text = text.lower()
    # Remove special characters and numbers
    text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Tokenize and remove stopwords
    try:
        stop_words = set(stopwords.words(lang))
    except OSError:
        print(f"Warning: '{lang}' stopwords not found. Defaulting to English.")
        stop_words = set(stopwords.words('english'))
    words = text.split()
    filtered_words = [word for word in words if word not in stop_words]
    # Join the words back into a single string
    return ' '.join(filtered_words)

# Function to generate sentiment labels using TextBlob
def generate_sentiment_labels(text):
    blob = TextBlob(text)
    # Classify sentiment as positive, negative, or neutral
    if blob.sentiment.polarity > 0:
        return 'positive'
    elif blob.sentiment.polarity < 0:
        return 'negative'
    else:
        return 'neutral'

# Function to load data, preprocess, and train the model
def train_and_evaluate_with_grid_search(file_path):
    # Load dataset
    df = pd.read_excel(file_path)  # Change to pd.read_excel for Excel files
    
    # Print column names to check if 'ulasan' exists
    print("Column names in the dataset:", df.columns)

    # Check if 'ulasan' column exists
    if 'ulasan' not in df.columns:
        print("Error: 'ulasan' column not found in the dataset. Please check the column names.")
        return

    # Handle missing values in 'ulasan' column
    df.dropna(subset=['ulasan'], inplace=True)

    # Generate sentiment labels based on 'ulasan' column
    print("Generating sentiment labels...")
    df['sentiment'] = df['ulasan'].apply(generate_sentiment_labels)

    # Preprocess the text data
    print("Preprocessing text...")
    df['ulasan'] = df['ulasan'].apply(lambda x: preprocess_text(x, lang='indonesian'))

    # Split dataset into features and labels
    X = df['ulasan']
    y = df['sentiment']

    # Vectorize the text using TF-IDF
    vectorizer = TfidfVectorizer(max_features=5000)
    X = vectorizer.fit_transform(X)

    # Handle class imbalance with SMOTE
    print("Handling class imbalance with SMOTE...")
    smote = SMOTE(random_state=42)
    X_resampled, y_resampled = smote.fit_resample(X, y)

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.25, random_state=42)

    # Initialize classifier
    clf = RandomForestClassifier(random_state=42)

    # Train the model
    print("Training model...")
    clf.fit(X_train, y_train)

    # Evaluate on training data
    y_train_pred = clf.predict(X_train)
    print("\nClassification Report (Training):")
    print(classification_report(y_train, y_train_pred))

    # Evaluate on testing data
    y_test_pred = clf.predict(X_test)
    print("\nClassification Report (Testing):")
    print(classification_report(y_test, y_test_pred))

    # Confusion Matrix
    print("\nConfusion Matrix (Testing):")
    print(confusion_matrix(y_test, y_test_pred))

    # Calculate additional metrics
    print("Performance Metrics (Testing):")
    print(f"Accuracy: {accuracy_score(y_test, y_test_pred)}")
    print(f"Precision: {precision_score(y_test, y_test_pred, average='weighted')}")
    print(f"Recall: {recall_score(y_test, y_test_pred, average='weighted')}")
    print(f"F1 Score: {f1_score(y_test, y_test_pred, average='weighted')}")

# File path to the dataset
file_path = "C:/Users/hp/Downloads/Semester 5/Scrape Sinergia/Shopee/clean ulasan face up saja shopee.xlsx"

# Run the function
train_and_evaluate_with_grid_search(file_path)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\hp\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Column names in the dataset: Index(['ulasan'], dtype='object')
Generating sentiment labels...
Preprocessing text...
Handling class imbalance with SMOTE...
Training model...

Classification Report (Training):
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00       492
     neutral       1.00      1.00      1.00       486
    positive       1.00      1.00      1.00       486

    accuracy                           1.00      1464
   macro avg       1.00      1.00      1.00      1464
weighted avg       1.00      1.00      1.00      1464


Classification Report (Testing):
              precision    recall  f1-score   support

    negative       1.00      1.00      1.00       159
     neutral       0.93      1.00      0.96       165
    positive       1.00      0.93      0.96       165

    accuracy                           0.98       489
   macro avg       0.98      0.98      0.98       489
weighted avg       0.98      0.98      0.98       4

In [12]:
pip install textblob

   ---------------------------------------- 0.0/626.3 kB ? eta -:--:--
   ---------------------------------------- 626.3/626.3 kB 7.7 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [14]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [15]:
pip install imbalanced-learn

Note: you may need to restart the kernel to use updated packages.
